# Netflix Investigation

In [ ]:
!pip install pymysql

In [2]:
import sqlalchemy as sal
import pandas as pd
from dotenv import load_dotenv
import os

In [4]:
load_dotenv(dotenv_path='../Keys/keys.env')

# Construct connection string based on database type
db_type = os.getenv('DB_TYPE').lower()

conn_string = (
    f"mysql+pymysql://{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}"
    f"@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}"
    f"/{os.getenv('DB_NAME')}"
)

# Create database engine
engine = sal.create_engine(conn_string)
conn = engine.connect()


In [6]:
# --- Call the create Schema file  ---
schema_file_path = '../SQL/create_netflix_titles.sql' 

try:
    with open(schema_file_path, 'r') as f:
        sql_commands = f.read()

    statements = [s.strip() for s in sql_commands.split(';') if s.strip()]

    with engine.connect() as connection:
        for statement in statements:
            if statement: # Ensure statement is not empty after stripping
                connection.execute(sal.text(statement)) # Use text() for literal SQL
        connection.commit() # Commit the DDL changes

    print(f"✅ Schema from {schema_file_path} executed successfully!")

except FileNotFoundError:
    print(f"❌ Error: Schema file not found at {schema_file_path}")
except Exception as e:
    print(f"❌ Error executing schema from {schema_file_path}:")
    print(e)


✅ Schema from ../SQL/create_netflix_titles.sql executed successfully!


In [8]:
df = pd.read_csv('../netflix_titles.csv')

In [10]:
# --- Push the Dataframe to the Schema  ---
try:
    df.to_sql(
        name='netflix_titles',  # table name in your database
        con=engine,              # SQLAlchemy engine
        if_exists='append',     # 'replace' to drop & recreate table, 'append' to add to it
        index=False              # don't write pandas DataFrame index as a column
    )
    print("✅ Data written to database successfully!")

except Exception as e:
    print("❌ Error writing DataFrame to database:")
    print(e)


✅ Data written to database successfully!


#### This recreates the table to have the correct sized columns and to handle foreign characters.

